In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import numpy as np
import json

In [2]:
df_net_worth = pd.read_csv('net_worth.csv', sep=';')
df_pre_tax_income = pd.read_csv('pre_tax_income.csv', sep=';')

In [3]:
countries_available = df_net_worth.country.unique()

In [4]:
with open('country_iso2_mapper.json', 'r') as f:
    country_iso2_mapper = json.load(f)

In [5]:
html = '''
<style>
    .imgbox {
        display: grid;
        height: 100%;
    }
    
    .center-fit {
        max-width: 100%;
        max-height: 100vh;
        margin: auto;
        margin-top: 20px;
    }
</style>

<h2>Inequality Worldwide</h2>
<p>The <a href="https://wid.world/" target="_blank">World Inequality Database provides data</a> for almost all countries in the world concerning:
<ul>
    <li>The net worth distribution for the top 1%, top 10% and bottom 50%</li>
    <li>The pre-tax income distribution of the top 1%, top 10% and bottom 50%</li>
    <li>The women's share of the pre-tax income distribution</li>
</ul>
of the respective country.</p>
<h2>Graphs</h2>
'''

In [6]:
for country in countries_available:
    # two subplots
    fig, axes = plt.subplots(1, 3, figsize=(20, 10))
    
    def plot_percentiles(df, country, ax, ylabel, percentiles_available, percentiles_titles):
        # show percentages on legend
        ax.yaxis.set_major_formatter(FuncFormatter(lambda y , _: '{:.0%}'.format(y)))
        # set max y should show 100%
        ax.set_ylim(0, 1)
        # show ygrid
        ax.yaxis.grid(True, linestyle='--', alpha=0.7)
        # plot all percentiles_available
        # with years on X and percentage values on Y
        for i in range(len(percentiles_available)):
            part = df[(df.country == country) & (df.percentile == percentiles_available[i])]
            ax.plot(part.year.values, part.value.values, label=percentiles_titles[i])
        # show the legend
        ax.legend()
        # set x label is years
        ax.set_xlabel('years')
        # and ylabel
        ax.set_ylabel(ylabel)
        
        
    # the 0 subplot is for the net worth of the percentiles
    # p0p50, p90p100, p99p100
    axes[0].set_title(f'net worth distribution in {country}')
    plot_percentiles(df_net_worth, country, axes[0],
                     f'% {country}\'s aggregate net worth',
                    ['p0p50', 'p90p100', 'p99p100'],
                    ['bottom 50%', 'top 10%', 'top 1%'])
    
    # the second subplot is for the pre-tax income of the same percentiles
    # of the respecive country
    axes[1].set_title(f'pre-tax income distribution in {country}')
    plot_percentiles(df_pre_tax_income, country, axes[1],
                     f'% {country}\'s aggregate pre-tax income',
                    ['p0p50', 'p90p100', 'p99p100'],
                    ['bottom 50%', 'top 10%', 'top 1%'])

    # the third subplot plots women's share of pre-tax income
    axes[2].set_title(f'pre-tax income distribution in {country}')
    # plot men's distribution (which is 1 - women's)
    part = df_pre_tax_income[(df_pre_tax_income.country == country) & (df_pre_tax_income.percentile == 'p0p100')]
    axes[2].plot(part.year.values, 1 - part.value.values, label='men')
    
    plot_percentiles(df_pre_tax_income, country, axes[2],
                     f'% {country}\'s aggregate pre-tax income',
                     ['p0p100'],
                     ['women'])


    # make tight layout
    plt.tight_layout()

    # save as file
    file_name = country_iso2_mapper[country] + '.png'
    plt.savefig('../public/inequality_graphs/' + file_name, dpi=100, bbox_inches='tight', format='png')

    # close figure
    plt.close()

    # and append to html (title and image)
    html += f'\n<h3>{country}</h3>'
    html += f'\n<div class="imgbox"><img src="/inequality_graphs/{file_name}" alt="{country} inequality graph" class="center-fit" onclick="enlargeImage(this)"></div><br><br><br>'

In [7]:
# write html to file
with open('../views/inequality-worldwide.hbs', 'w') as f:
    f.write(html)